# 3. Training Pipeline

## 3.1. Setup

In [3]:
# Standard imports
import os
import sys
import json
import time
from datetime import date, datetime, timedelta
import warnings
from pathlib import Path
warnings.filterwarnings("ignore", module="IPython")

#  Establish project root directory
def find_project_root(start: Path):
    for parent in [start] + list(start.parents):
        if (parent / "pyproject.toml").exists():
            return parent
    return start

root_dir = find_project_root(Path().absolute())
print("Project root dir:", root_dir)

if str(root_dir) not in sys.path:
    sys.path.append(str(root_dir))

# Third-party imports
import requests
import pandas as pd
import great_expectations as gx
import hopsworks
from urllib3.exceptions import ProtocolError
from requests.exceptions import ConnectionError, Timeout, RequestException
from collections import defaultdict
import matplotlib.pyplot as plt
from xgboost import XGBRegressor
from xgboost import plot_importance
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

#  Project imports
from utils import cleaning, config, feature_engineering, fetchers, hopsworks_admin, incremental, metadata

#  Load settings 
settings = config.HopsworksSettings(_env_file=f"{root_dir}/.env")
HOPSWORKS_API_KEY = settings.HOPSWORKS_API_KEY.get_secret_value()
GITHUB_USERNAME = settings.GH_USERNAME.get_secret_value()

# Login to Hopsworks
project = hopsworks.login(api_key_value=HOPSWORKS_API_KEY)
fs = project.get_feature_store()

Project root dir: c:\Users\krist\Documents\GitHub\pm25
HopsworksSettings initialized!
2026-01-14 13:46:51,850 INFO: Closing external client and cleaning up certificates.
2026-01-14 13:46:51,850 INFO: Initializing external client
2026-01-14 13:46:51,850 INFO: Base URL: https://c.app.hopsworks.ai:443
2026-01-14 13:46:53,498 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1279184


Repository

In [4]:
repo_dir = hopsworks_admin.clone_or_update_repo(GITHUB_USERNAME)
os.chdir(repo_dir)

Repository exists at c:\Users\krist\Documents\GitHub\pm25\notebooks\pm25-forecast-openmeteo-aqicn


## 3.2. Get secrets

In [5]:
today = date.today()

if settings.AQICN_API_KEY is None:
    print("AQICN_API_KEY missing.")
    sys.exit(1)

AQICN_API_KEY = settings.AQICN_API_KEY.get_secret_value()

secrets = hopsworks.get_secrets_api()
try:
    secret = secrets.get_secret("AQICN_API_KEY")
    if secret is not None:
        secret.delete()
except Exception:
    pass

secrets.create_secret("AQICN_API_KEY", AQICN_API_KEY)

Secret created successfully, explore it at https://c.app.hopsworks.ai:443/account/secrets


Secret('AQICN_API_KEY', 'PRIVATE')

In [6]:
secrets = hopsworks.get_secrets_api()
AQICN_API_KEY = secrets.get_secret("AQICN_API_KEY").value

today = datetime.today().date()

# Retrieve feature groups
air_quality_fg = fs.get_feature_group(
    name="air_quality",
    version=1,
)

weather_fg = fs.get_feature_group(
    name="weather",
    version=1,
)

## 3.3. Get Feature Groups

In [7]:
air_quality_fg, sensor_metadata_fg, weather_fg = hopsworks_admin.create_feature_groups(fs)

## 3.4. Load Metadata from Feature Group

In [8]:
metadata_df = sensor_metadata_fg.read()
if len(metadata_df) == 0:
    print("⚠️ No sensor metadata found. Run pipeline 1 (backfill) first.")
    sys.exit(1)

print(f"📍 Loaded metadata for {len(metadata_df)} sensors")
metadata_df = metadata_df.set_index("sensor_id")

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.96s) 
📍 Loaded metadata for 103 sensors


## 3.5. Feature View Creation
Create multiple feature views with different feature combinations (baseline, rolling windows, lagged features, nearby sensors, complete) for model comparison.

In [9]:
# from hsfs.feature_view import FeatureView

# fs_id = fs.id

# fv_names = [
#     "air_quality_baseline_fv",
#     "air_quality_rolling_fv",
#     "air_quality_nearby_fv",
#     "air_quality_lagged_1d_fv",
#     "air_quality_lagged_2d_fv",
#     "air_quality_lagged_3d_fv",
#     "air_quality_complete_fv",
# ]

# for name in fv_names:
#     for version in range(1, 10):  # delete versions 1–9
#         try:
#             FeatureView.clean(fs_id, name, version)
#             print(f"Deleted {name} v{version}")
#         except Exception as e:
#             if "wasn't found" in str(e):
#                 continue
#             else:
#                 print(f"Unexpected error cleaning {name} v{version}: {e}")

In [10]:
# Select features for training data
baseline_features = air_quality_fg.select(["pm25", "date", "sensor_id", "location_id"]).join(
    weather_fg.select_features(), on=["location_id", "date"])

baseline_feature_view = fs.get_or_create_feature_view(
    name="air_quality_baseline_fv",
    description="Weather features for PM2.5 prediction",
    version=1,
    labels=["pm25"],
    query=baseline_features,
)

rolling_features = air_quality_fg.select(["pm25", "pm25_rolling_3d", "date", "sensor_id", "location_id"]).join(
    weather_fg.select_features(), on=["location_id", "date"])
rolling_feature_view = fs.get_or_create_feature_view(
    name="air_quality_rolling_fv",
    description="Weather features, PM2.5 rolling window (3d) for PM2.5 prediction",
    version=1,
    labels=["pm25"],
    query=rolling_features,
)

nearby_features = air_quality_fg.select(["pm25", "pm25_nearby_avg", "date", "sensor_id", "location_id"]).join(
    weather_fg.select_features(), on=["location_id", "date"])
nearby_feature_view = fs.get_or_create_feature_view(
    name="air_quality_nearby_fv",
    description="Weather features, PM2.5 nearby average (1d lag, 3 sensors) for PM2.5 prediction",
    version=1,
    labels=["pm25"],
    query=nearby_features,
)

lagged_1d_features = air_quality_fg.select(["pm25", "pm25_lag_1d", "date", "sensor_id", "location_id"]).join(
    weather_fg.select_features(), on=["location_id", "date"])
lagged_1d_feature_view = fs.get_or_create_feature_view(
    name="air_quality_lagged_1d_fv",
    description="Weather features, PM2.5 lags (1d) for PM2.5 prediction",
    version=1,
    labels=["pm25"],
    query=lagged_1d_features,
)

lagged_2d_features = air_quality_fg.select(["pm25", "pm25_lag_1d", "pm25_lag_2d", "date", "sensor_id", "location_id"]).join(
    weather_fg.select_features(), on=["location_id", "date"])
lagged_2d_feature_view = fs.get_or_create_feature_view(
    name="air_quality_lagged_2d_fv",
    description="Weather features, PM2.5 lags (1d, 2d) for PM2.5 prediction",
    version=1,
    labels=["pm25"],
    query=lagged_2d_features,
)

lagged_3d_features = air_quality_fg.select(["pm25", "pm25_lag_1d", "pm25_lag_2d", "pm25_lag_3d", "date", "sensor_id", "location_id"]).join(
    weather_fg.select_features(), on=["location_id", "date"])
lagged_3d_feature_view = fs.get_or_create_feature_view(
    name="air_quality_lagged_3d_fv",
    description="Weather features, PM2.5 lags (1d, 2d, 3d) for PM2.5 prediction",
    version=1,
    labels=["pm25"],
    query=lagged_3d_features,
)

complete_features = air_quality_fg.select(["pm25", "pm25_rolling_3d", "pm25_lag_1d", "pm25_lag_2d", "pm25_lag_3d", "pm25_nearby_avg", "date", "sensor_id", "location_id"]).join(
    weather_fg.select_features(), on=["location_id", "date"])
complete_feature_view = fs.get_or_create_feature_view(
    name="air_quality_complete_fv",
    description="Weather features, PM2.5 rolling window (3d), and PM2.5 lags (1d, 2d, 3d), and PM2.5 nearby average (1d lag, 3 sensors) for PM2.5 prediction",
    version=1,
    labels=["pm25"],
    query=complete_features,
)

2026-01-14 13:47:02,754 INFO: Using ['temperature_2m_mean', 'precipitation_sum', 'wind_speed_10m_max', 'wind_direction_10m_dominant'] from feature group `weather` as features for the query. To include primary key and event time use `select_all`.
2026-01-14 13:47:04,748 INFO: Using ['temperature_2m_mean', 'precipitation_sum', 'wind_speed_10m_max', 'wind_direction_10m_dominant'] from feature group `weather` as features for the query. To include primary key and event time use `select_all`.
2026-01-14 13:47:06,702 INFO: Using ['temperature_2m_mean', 'precipitation_sum', 'wind_speed_10m_max', 'wind_direction_10m_dominant'] from feature group `weather` as features for the query. To include primary key and event time use `select_all`.
2026-01-14 13:47:08,694 INFO: Using ['temperature_2m_mean', 'precipitation_sum', 'wind_speed_10m_max', 'wind_direction_10m_dominant'] from feature group `weather` as features for the query. To include primary key and event time use `select_all`.
2026-01-14 13:47

## 3.6. Model Training Setup
Set up test data split date, initialize containers for models and predictions, and define feature view dictionary for iteration

In [11]:
models = defaultdict(dict)
y_preds = defaultdict(dict)
results = []

feature_views = {
    "baseline": baseline_feature_view,
    "rolling": rolling_feature_view,
    "nearby": nearby_feature_view,
    "lagged_1d": lagged_1d_feature_view,
    "lagged_2d": lagged_2d_feature_view,
    "lagged_3d": lagged_3d_feature_view,
    "complete": complete_feature_view,
}

## 3.7. Model Training Loop
Train XGBoost models for each feature combination and sensor, run 5 iterations per configuration, select best model based on R2 score, and store results.

In [12]:
# Should all of these be saved to model registry instead???
model_dir = f"{root_dir}/models"
if not os.path.exists(model_dir):
    os.mkdir(model_dir)

In [13]:
# calculate the mean for weather features for each day and location_id and replace in fg
weather_daily_mean = (
    weather_fg.read()
    .groupby(["location_id", "date"])
    .mean()
    .reset_index()
)

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (6.80s) 


## 3.8. Debug data quality
To store trained models

In [14]:
# #  DEBUG:
# print("🔍 DEBUGGING DATA QUALITY")
# print("="*60)

# # Check one sensor's data
# first_feature_name = list(feature_views.keys())[0]
# sample_data = feature_views[first_feature_name].query.read()
# test_sensor_id = sample_data["sensor_id"].iloc[0]

# print(f"\n📊 Checking sensor {test_sensor_id}:")

# for feature_name, feature_view in feature_views.items():
#     data = feature_view.query.read()
#     sensor_data = data[data["sensor_id"] == test_sensor_id].copy()
    
#     if len(sensor_data) == 0:
#         print(f"  {feature_name}: NO DATA")
#         continue
    
#     print(f"\n  {feature_name}: {len(sensor_data)} rows")
#     print(f"    Date range: {sensor_data['date'].min()} to {sensor_data['date'].max()}")
    
#     # Check each column for NaN
#     for col in sensor_data.columns:
#         nan_count = sensor_data[col].isna().sum()
#         nan_pct = (nan_count / len(sensor_data)) * 100
#         if nan_pct > 0:
#             print(f"    {col}: {nan_count}/{len(sensor_data)} NaN ({nan_pct:.1f}%)")
    
#     # Show first few rows
#     print(f"\n    Sample data:")
#     print(sensor_data.head())
#     break  # Only check baseline for now

# print("\n" + "="*60)

Training loop

In [15]:
# print(feature_name, data.shape)
# print(data["sensor_id"].unique()[:10])

In [16]:
# print("=== FEATURE VIEW CHECK ===")
# for feature_name, feature_view in feature_views.items():
#     data = feature_view.query.read()
#     print(feature_name, data.shape)
#     print("unique sensors:", len(data["sensor_id"].unique()))
#     print("sample sensors:", data["sensor_id"].unique()[:5])
#     print()

In [17]:
# for feature_name, feature_view in feature_views.items():
#     data = feature_view.query.read()
#     data['date'] = pd.to_datetime(data['date']).dt.tz_localize(None)

#     for sensor_id in data["sensor_id"].unique():
#         df = data[data["sensor_id"] == sensor_id].copy()
#         print(df.isna().sum())

In [18]:
models = {name: {} for name in feature_views.keys()}
y_preds = {name: {} for name in feature_views.keys()}
results = []

In [19]:
for feature_name, feature_view in feature_views.items():
    data = feature_view.query.read()
    data['date'] = pd.to_datetime(data['date']).dt.tz_localize(None)

    for sensor_id in data["sensor_id"].unique():

    # for sensor_id in metadata_df.index:
        df = data[data["sensor_id"] == sensor_id].copy()

        # Only drop rows where pm25 (target) is NaN
        df = df.dropna(subset=["pm25"])
        
        if len(df) < 10:
            print(f"⚠️  Skipping sensor {sensor_id} ({feature_name}): insufficient data ({len(df)} rows)")
            continue

        # Identify which features this feature view actually uses
        # Drop non-feature columns first
        feature_cols = [c for c in df.columns if c not in ["pm25", "date", "sensor_id", "location_id"]]
        
        # Keep rows with NaN in features - XGBoost handles NaN natively
        df_clean = df.copy()
        
        if len(df_clean) < 10:
            print(f"⚠️  Skipping sensor {sensor_id} ({feature_name}): insufficient clean data ({len(df_clean)} rows, needed features: {feature_cols})")
            continue

        # Split
        train_size = int(0.8 * len(df_clean))
        train_df = df_clean.iloc[:train_size]
        test_df = df_clean.iloc[train_size:]

        if len(test_df) < 2:
            print(f"⚠️  Skipping sensor {sensor_id} ({feature_name}): test set too small ({len(test_df)} rows)")
            continue

        # Prepare features and target
        X_train = train_df[feature_cols]
        y_train = train_df["pm25"]
        X_test = test_df[feature_cols]
        y_test = test_df["pm25"]

        best_r2 = -float('inf')
        best_mse = float('inf')
        best_model = None

        mse_list = []
        r2_list = []
        
        for i in range(5):
            model = XGBRegressor(n_estimators=100, learning_rate=0.05, random_state=165439*i)
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)
            r2 = r2_score(y_test, y_pred)
            mse = mean_squared_error(y_test, y_pred)
            r2_list.append(r2)
            mse_list.append(mse)
            
            if r2 > best_r2:
                best_r2 = r2
                best_mse = mse
                best_model = model

        models[feature_name][sensor_id] = best_model

        if best_model is not None:
            y_preds[feature_name][sensor_id] = best_model.predict(X_test)
            results.append({
                "feature_name": feature_name,
                "sensor_id": sensor_id,
                "MSE": sum(mse_list) / len(mse_list),
                "R2": sum(r2_list) / len(r2_list),
                "train_size": len(X_train),
                "test_size": len(X_test),
            })
            print(f"✅ {feature_name} - Sensor {sensor_id}: R2={best_r2:.3f}, MSE={best_mse:.2f}")
        else:
            print(f"⚠️  No valid model for {feature_name} - {sensor_id}")

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (10.74s) 
2026-01-14 13:47:37,785 WARNING: DeprecationWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.

2026-01-14 13:47:37,785 WARNING: DeprecationWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.

2026-01-14 13:47:37,787 WARNING: DeprecationWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.

2026-01-14 13:47:37,787 WARNING: DeprecationWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.

2026-01-14 13:47:37,870 WARNING: DeprecationWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.

2026-01-14 13:47:37,872 WARNING: DeprecationWarning: is_sparse 

In [20]:
# for feature_name, feature_view in feature_views.items():
#     data = feature_view.query.read()
#     data['date'] = pd.to_datetime(data['date']).dt.tz_localize(None)

#     for sensor_id in metadata_df.index:
#         df = data[data["sensor_id"] == sensor_id].copy()

#         # Clean the data before splitting
#         df = df.dropna(subset=["pm25"])
#         # features_for_cleaning = df.drop(columns=["pm25", "date", "city", "sensor_id"])
#         features_for_cleaning = df.drop(columns=["pm25", "date", "sensor_id"])

#         target_for_cleaning = df["pm25"]
#         clean_mask = ~(features_for_cleaning.isna().any(axis=1) | target_for_cleaning.isna())
#         df_clean = df[clean_mask].copy()
#         if len(df_clean) < 10:
#             print(f"⚠️  Skipping sensor {sensor_id}: insufficient data after cleaning ({len(df_clean)} rows)")
#             continue

#         train_size = int(0.8 * len(df_clean))
#         train_df = df_clean.iloc[:train_size]
#         test_df = df_clean.iloc[train_size:]

#         if len(test_df) < 2:
#             print(f"⚠️  Skipping sensor {sensor_id}: test set too small after split ({len(test_df)} rows)")
#             continue

#         # Drop non-feature columns (pm25 is target, others are metadata)
#         X_train = train_df.drop(columns=["pm25", "date", "city", "sensor_id"])
#         y_train = train_df["pm25"]
#         X_test = test_df.drop(columns=["pm25", "date", "city", "sensor_id"])
#         y_test = test_df["pm25"]

#         best_r2 = -float('inf')
#         best_mse = float('inf')
#         best_model = None

#         mse_list = []
#         r2_list = []
#         for i in range(5):
#             model = XGBRegressor(n_estimators=100, learning_rate=0.05, random_state=165439*i)
#             model.fit(X_train, y_train)
#             y_pred = model.predict(X_test)
#             r2 = r2_score(y_test, y_pred)
#             mse = mean_squared_error(y_test, y_pred)
#             r2_list.append(r2)
#             mse_list.append(mse)
#             if r2 > best_r2:
#                 best_r2 = r2
#                 best_mse = mse
#                 best_model = model

#         models[feature_name][sensor_id] = best_model

#         if best_model is not None:
#             y_preds[feature_name][sensor_id] = best_model.predict(X_test)
#             results.append({
#                 "feature_name": feature_name,
#                 "sensor_id": sensor_id,
#                 "MSE": sum(mse_list) / len(mse_list),
#                 "R2": sum(r2_list) / len(r2_list),
#                 "train_size": len(X_train),
#                 "test_size": len(X_test),
#             })
#         else:
#             print(f"⚠️  No valid model trained for {feature_name} - {sensor_id}, R2 scores: {r2_list}, Best R2: {best_r2}")
            

## 3.9. Model Selection and Saving
Identify best performing model for each sensor, save models and feature importance plots, and prepare test data with predictions.

In [25]:
print(metadata_df.head())
print(metadata_df.index.name)
print(metadata_df.columns)

                   city              street country  \
sensor_id                                             
191047      Nödinge-Nol  Gamla Kilandavägen  Sweden   
59410       Ringvattnet            Z 811.01  Sweden   
113542       Sollentuna           Götavägen  Sweden   
59497         Österskär          Dadelvägen  Sweden   
420664     Skåne County      Östra Odarslöv  Sweden   

                                     aqicn_url   latitude  longitude  \
sensor_id                                                              
191047     https://api.waqi.info/feed/A191047/  57.882784  12.094796   
59410       https://api.waqi.info/feed/A59410/  64.183610  15.423732   
113542     https://api.waqi.info/feed/A113542/  59.419607  17.933547   
59497       https://api.waqi.info/feed/A59497/  59.458142  18.310849   
420664     https://api.waqi.info/feed/A420664/  55.739472  13.276084   

           location_id  
sensor_id               
191047              15  
59410               49  
113542   

In [ ]:
# --- Find best model per sensor ---
results_df = pd.DataFrame(results)
best_models = results_df.loc[results_df.groupby("sensor_id")["R2"].idxmax()]

print("Best models per sensor:")
print(best_models[["sensor_id", "feature_name", "R2", "MSE"]])

# --- Cache all feature view data once ---
cached_feature_data = {
    name: fv.query.read()
    for name, fv in feature_views.items()
}

# Normalize dates in cached data
for name, df_cached in cached_feature_data.items():
    df_cached["date"] = pd.to_datetime(df_cached["date"]).dt.tz_localize(None)

all_test_data = []

# --- Loop through best models ---
for _, row in best_models.iterrows():
    sensor_id = row["sensor_id"]
    best_feature = row["feature_name"]

    # --- Save model + feature importance ---
    sensor_dir = f"{model_dir}/{sensor_id}"
    images_dir = f"{sensor_dir}/images"
    os.makedirs(images_dir, exist_ok=True)

    best_model = models[best_feature][sensor_id]
    best_model.save_model(f"{sensor_dir}/model.json")

    plot_importance(best_model)
    plt.savefig(f"{images_dir}/feature_importance.png")
    plt.close()

    # --- Load cached feature view data ---
    df = cached_feature_data[best_feature]
    df = df[df["sensor_id"] == sensor_id].copy()

    # --- EXACT same cleaning as training ---
    df = df.sort_values("date").reset_index(drop=True)
    df = df.dropna(subset=["pm25"])

    feature_cols = [
        c for c in df.columns
        if c not in ["pm25", "date", "sensor_id", "location_id"]
    ]

    # Keep rows with NaN in features - same as training (XGBoost handles NaN)
    df_clean = df.copy()
    if len(df_clean) < 10:
        continue

    # --- EXACT same split as training ---
    train_size = int(0.8 * len(df_clean))
    test_df = df_clean.iloc[train_size:].copy()
    if len(test_df) == 0:
        continue

    # --- Predictions from training ---
    predictions = y_preds[best_feature][sensor_id]

    # --- Match lengths ---
    if len(predictions) != len(test_df):
        print(f"⚠️ Skipping sensor {sensor_id}: prediction length mismatch "
              f"({len(predictions)} vs {len(test_df)})")
        continue

    # --- Store aligned test data ---
    test_df["predicted_pm25"] = predictions
    test_df["best_model"] = best_feature

    metadata_coord_df = metadata_df.set_index("sensor_id")

    lat = metadata_coord_df.at[sensor_id, "latitude"]
    lon = metadata_coord_df.at[sensor_id, "longitude"]
    test_df["latitude"] = lat
    test_df["longitude"] = lon

    all_test_data.append(
        test_df[["date", "pm25", "predicted_pm25", "latitude", "longitude", "best_model"]]
    )

Best models per sensor:
     sensor_id feature_name        R2       MSE
660      57421     complete  0.430189  8.766714
558      58666    lagged_3d  0.268970  8.940198
353      58909    lagged_1d  0.427671  4.321312
560      58912    lagged_3d  0.436510  3.958336
467      58921    lagged_2d  0.919309  0.053052
..         ...          ...       ...       ...
511     494275    lagged_2d  0.696446  1.224997
656     497266     complete  0.644419  2.936786
512     533086    lagged_2d  0.426032  5.314410
513     556792    lagged_2d  0.243221  2.819133
350     562600    lagged_1d  0.444209  7.488484

[103 rows x 4 columns]
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (9.65s) 
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (9.65s) 
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (7.91s) 
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (6.85s) 
Finished: Reading data from Hops

NameError: name 'metadata_ccord_df' is not defined

In [ ]:
# # Find best model (highest R2) for each sensor
# results_df = pd.DataFrame(results)
# best_models = results_df.loc[results_df.groupby('sensor_id')['R2'].idxmax()]
# best_models = results_df.loc[results_df.groupby('sensor_id')['R2'].idxmax()]

# print("Best models per sensor:")
# print(best_models[['sensor_id', 'feature_name', 'R2', 'MSE']])

# all_data = baseline_features.read()
# all_data['date'] = pd.to_datetime(all_data['date']).dt.tz_localize(None)

# cached_feature_data = {
#     name: fv.query.read()
#     for name, fv in feature_views.items()
# }

# all_test_data = []

# for _, row in best_models.iterrows():
#     sensor_id = row['sensor_id']
#     best_feature = row['feature_name']
    
#     sensor_dir = f"{model_dir}/{sensor_id}"
#     if not os.path.exists(sensor_dir):
#         os.mkdir(sensor_dir)
#     images_dir = f"{model_dir}/{sensor_id}/images"
#     if not os.path.exists(images_dir):
#         os.mkdir(images_dir)

#     best_model = models[best_feature][sensor_id]
#     model_path = f"{sensor_dir}/model.json"
#     plot_importance(best_model)
#     importance_path = f"{images_dir}/feature_importance.png"
#     plt.savefig(importance_path)
#     plt.close()
    
#     best_model.save_model(model_path)

#     # Use the same feature view and data processing logic that was used for training
#     best_feature_view = feature_views[best_feature]
#     sensor_data = cached_feature_data[best_feature]
#     sensor_data['date'] = pd.to_datetime(sensor_data['date']).dt.tz_localize(None)
    
#     df = sensor_data[sensor_data['sensor_id'] == sensor_id].copy()
    
#     df = df.sort_values("date").reset_index(drop=True)

#     # Apply EXACT same cleaning logic as in training loop
#     df = df.dropna(subset=['pm25'])
    
#     # Recompute feature columns exactly as in training
#     feature_cols = [c for c in df.columns if c not in ["pm25", "date", "sensor_id", "location_id"]]

#     # Drop rows with NaN in any feature
#     df_clean = df.dropna(subset=feature_cols).copy()

#     if len(df_clean) < 10:
#         continue

#     # Same split as training
#     train_size = int(0.8 * len(df_clean))
#     test_df = df_clean.iloc[train_size:].copy()

#     # # Create feature matrix for comprehensive NaN cleaning (same as training)
#     # features_for_cleaning = df.drop(columns=["pm25", "date", "sensor_id"])
#     # target_for_cleaning = df["pm25"]
    
#     # # Remove rows with NaN values in any feature or target (same as training)
#     # clean_mask = ~(features_for_cleaning.isna().any(axis=1) | target_for_cleaning.isna())
#     # df_clean = df[clean_mask].copy()
    
#     # if len(df_clean) < 10:
#     #     continue
        
#     # # Split the cleaned data (same as training)
#     # train_size = int(0.8 * len(df_clean))
#     # test_df = df_clean.iloc[train_size:].copy()
    
#     if len(test_df) == 0:
#         continue
    
#     # Test data is already clean from the comprehensive cleaning above
#     clean_test_df = test_df.copy()
#     predictions = y_preds[best_feature][sensor_id]
    
#     if len(clean_test_df) == len(predictions):
#         clean_test_df['predicted_pm25'] = predictions
#         clean_test_df['best_model'] = best_feature
#         all_test_data.append(clean_test_df[['date', 'pm25', 'predicted_pm25', 'latitude', 'longitude', 'best_model']])
#     else:
#         print(f"⚠️  Skipping sensor {sensor_id}: prediction length mismatch ({len(predictions)} vs {len(clean_test_df)})")

## 3.10. Model Registration & Visualization
Create prediction plots for each sensor, register model in Hopsworks model registry with metrics and metadata, and save models with their configuration.

In [ ]:
mr = project.get_model_registry()
df = pd.concat(all_test_data, ignore_index=True) if all_test_data else pd.DataFrame()
df = df.sort_values(by=["date"])

# Plot the best model for each sensor
for sensor_id, meta in metadata_df.iterrows():
    city = meta["city"]
    street = meta["street"]
    latitude = meta["latitude"]
    longitude = meta["longitude"]
    
    df_subset = df[(df["latitude"] == latitude) & (df["longitude"] == longitude)].copy()
    if len(df_subset) == 0:
        continue
    
    # Get the best model name for display
    best_model_name = df_subset['best_model'].iloc[0] if 'best_model' in df_subset.columns else 'unknown'
    best_model_r2 = df_subset['R2'].iloc[0] if 'R2' in df_subset.columns else 0
    best_model_mse = df_subset['MSE'].iloc[0] if 'MSE' in df_subset.columns else 0
    best_model_feature_view = feature_views[best_model_name]
    
    df_subset = df_subset.sort_values(by=["date"])
    df_subset = df_subset.drop(columns=["latitude", "longitude", "best_model"])
    
    images_dir = f"{model_dir}/{sensor_id}/images"
    image_path = f"{images_dir}/hindcast_training.png"
    
    plt = airquality.plot_air_quality_forecast(
        city, street, df_subset, image_path, hindcast=True
    )
    plt.title(f"{city} {street} (Best Model: {best_model_name})")

    aq_model = mr.python.create_model(
        name=f"air_quality_xgboost_model_{sensor_id}",
        metrics={
            "R2": best_model_r2,
            "MSE": best_model_mse,
        },
        feature_view=best_model_feature_view,
        description=f"Air Quality (PM2.5) predictor for {city} {street} using {best_model_name} configuration",
    )

    aq_model.save(f"{model_dir}/{sensor_id}")